In [3]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import pandas as pd
import os
import random

os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [4]:
OriginData = np.loadtxt("FTIR Top Data.txt")

FTIR_FullSpectrum = OriginData[:,1]

num_steps = 1000
display_step = 1000

FTIR_FullSpectrum

learning_rate = 0.00001

n_InputUnit = 1738
n_1stHiddenUnit = 869
n_2ndHiddenUnit = 434
n_3rdHiddenUnit = 210

E_W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
E_W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
E_W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))

D_W1 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
D_W2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_1stHiddenUnit],stddev=0.01))
D_W3 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_InputUnit],stddev=0.01))


E_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
E_b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit],stddev=0.01))
E_b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit],stddev=0.01))

D_b1 = tf.Variable(tf.random_normal([n_2ndHiddenUnit],stddev=0.01))
D_b2 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
D_b3 = tf.Variable(tf.random_normal([n_InputUnit],stddev=0.01))

X = tf.placeholder(tf.float32, [None, n_InputUnit]) # tensorflow의 placeholder라는 변수를 선언
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    # Encoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, E_W1), E_b1))
    layer_2 = tf.nn.sigmoid(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    layer_3 = tf.nn.sigmoid(tf.add( tf.matmul(layer_2, E_W3), E_b3))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    return layer_3

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    # Decoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, D_W1), D_b1))
    layer_2 = tf.nn.sigmoid(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    layer_3 = tf.nn.sigmoid(tf.add( tf.matmul(layer_2, D_W3), D_b3))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    return layer_3

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.5).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

k=0

In [5]:
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738)) #25개
        
        _, aa = sess.run([optimizer, loss], feed_dict={X: batch})
        
        k = k+1
        if k == 25:
            k=0
            
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, aa))
        if i == num_steps:
            batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738))
            e = sess.run(encoder_op, feed_dict = {X: batch})
            g = sess.run(decoder_op, feed_dict={X: batch})
            print(i)
            print("batch")
            print(batch)
            print("decoded")
            print(g)
            print("encoded")
            print(e)
            print("----------------------------------------------")
    ###################################################
    print("######################################################")
    print("######################################################")
    for i in range(25):
        batch = np.reshape(FTIR_FullSpectrum[1738*(i) : 1738*(i+1)],(1,1738))
        g = sess.run(decoder_op, feed_dict={X: batch})
        print(i)
        print("batch")
        print(batch)
        print("decoded")
        print(g)
        print("encoded")
        print(e)
        print("----------------------------------------------")

Step 1: Minibatch Loss: 0.010101
Step 1000: Minibatch Loss: 0.000092
1000
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.34309754  0.34183228  0.34370729 ...,  0.5514009   0.55146867
   0.55158812]]
encoded
[[ 0.49363557  0.51483822  0.54119843  0.44188333  0.46162674  0.52921605
   0.56838816  0.50771064  0.49833521  0.50992215  0.50297594  0.48179546
   0.47664154  0.49111593  0.49660242  0.45716339  0.53034467  0.45106497
   0.47961387  0.48836908  0.52843314  0.47968683  0.5115009   0.44991767
   0.48609424  0.53954417  0.48093203  0.5044964   0.48682579  0.50986648
   0.48048341  0.46873656  0.50708961  0.52785361  0.53658402  0.48545989
   0.51652974  0.51974279  0.49343348  0.53931916  0.4640381   0.48658377
   0.48150739  0.46231604  0.49829766  0.52454233  0.520105    0.48590907
   0.47917742  0.5100674   0.49663416  0.50461423  0.48914158  0.55019099
   0.52758622  0.49761006  0.51076633  0.49364999  0.51142406  0.48509461
   0.4996847

In [6]:
OriginData = np.loadtxt("FTIR Top Data.txt")

FTIR_FullSpectrum = OriginData[:,1]

num_steps = 1000
display_step = 1000

FTIR_FullSpectrum

learning_rate = 0.00001

n_InputUnit = 1738
n_1stHiddenUnit = 869
n_2ndHiddenUnit = 434

E_W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
E_W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
D_W1 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_1stHiddenUnit],stddev=0.01))
D_W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_InputUnit],stddev=0.01))

E_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
E_b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit],stddev=0.01))
D_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
D_b2 = tf.Variable(tf.random_normal([n_InputUnit],stddev=0.01))

X = tf.placeholder(tf.float32, [None, n_InputUnit]) # tensorflow의 placeholder라는 변수를 선언
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    # Encoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, E_W1), E_b1))
    layer_2 = tf.nn.sigmoid(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    return layer_2

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    # Decoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, D_W1), D_b1))
    layer_2 = tf.nn.sigmoid(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    return layer_2

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.5).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

k=0

In [7]:
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738)) #25개
        
        _, aa = sess.run([optimizer, loss], feed_dict={X: batch})
        
        k = k+1
        if k == 25:
            k=0
            
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, aa))
        if i == num_steps:
            batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738))
            e = sess.run(encoder_op, feed_dict = {X: batch})
            g = sess.run(decoder_op, feed_dict={X: batch})
            print(i)
            print("batch")
            print(batch)
            print("decoded")
            print(g)
            print("encoded")
            print(e)
            print("----------------------------------------------")
    ###################################################
    print("######################################################")
    print("######################################################")
    for i in range(25):
        batch = np.reshape(FTIR_FullSpectrum[1738*(i) : 1738*(i+1)],(1,1738))
        g = sess.run(decoder_op, feed_dict={X: batch})
        print(i)
        print("batch")
        print(batch)
        print("decoded")
        print(g)
        print("encoded")
        print(e)
        print("----------------------------------------------")

Step 1: Minibatch Loss: 0.009884
Step 1000: Minibatch Loss: 0.000092
1000
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.3430846   0.3418299   0.34370619 ...,  0.55140233  0.55147827
   0.55159414]]
encoded
[[ 0.47759947  0.5123899   0.51333004  0.46715105  0.52824748  0.42258522
   0.49175721  0.53599745  0.48651329  0.5198316   0.51950693  0.50971431
   0.4550972   0.45776942  0.45509964  0.54362553  0.49364394  0.55197114
   0.53917712  0.45311865  0.49037147  0.50136298  0.49016792  0.52917391
   0.45726964  0.40356177  0.50820011  0.48465201  0.50346476  0.4941335
   0.51618558  0.46496642  0.49536449  0.45280516  0.49108708  0.55629265
   0.48961836  0.46081835  0.43253246  0.47018561  0.46248803  0.5157513
   0.51659179  0.58565336  0.55184853  0.51107967  0.46376365  0.50425029
   0.49322093  0.46058065  0.52209258  0.48054221  0.51435703  0.49844524
   0.47800776  0.46702445  0.41294315  0.46779132  0.56333351  0.48598233
   0.51991701 

[[ 0.47759947  0.5123899   0.51333004  0.46715105  0.52824748  0.42258522
   0.49175721  0.53599745  0.48651329  0.5198316   0.51950693  0.50971431
   0.4550972   0.45776942  0.45509964  0.54362553  0.49364394  0.55197114
   0.53917712  0.45311865  0.49037147  0.50136298  0.49016792  0.52917391
   0.45726964  0.40356177  0.50820011  0.48465201  0.50346476  0.4941335
   0.51618558  0.46496642  0.49536449  0.45280516  0.49108708  0.55629265
   0.48961836  0.46081835  0.43253246  0.47018561  0.46248803  0.5157513
   0.51659179  0.58565336  0.55184853  0.51107967  0.46376365  0.50425029
   0.49322093  0.46058065  0.52209258  0.48054221  0.51435703  0.49844524
   0.47800776  0.46702445  0.41294315  0.46779132  0.56333351  0.48598233
   0.51991701  0.47291297  0.52173811  0.46531916  0.52767539  0.47466975
   0.48327899  0.47230473  0.44614896  0.45827642  0.48870763  0.49270841
   0.46792528  0.43579802  0.54218972  0.49043483  0.47525403  0.45614016
   0.60462129  0.51056355  0.50452322  0

In [8]:
OriginData = np.loadtxt("FTIR Top Data.txt")

FTIR_FullSpectrum = OriginData[:,1]

num_steps = 1000
display_step = 1000

FTIR_FullSpectrum

learning_rate = 0.00001

n_InputUnit = 1738
n_1stHiddenUnit = 869
n_2ndHiddenUnit = 434
n_3rdHiddenUnit = 210
n_4thHiddenUnit = 100

E_W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
E_W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
E_W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
E_W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))

D_W1 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
D_W2 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
D_W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_1stHiddenUnit],stddev=0.01))
D_W4 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_InputUnit],stddev=0.01))

########################################################################################

E_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
E_b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit],stddev=0.01))
E_b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit],stddev=0.01))
E_b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit],stddev=0.01))

D_b1 = tf.Variable(tf.random_normal([n_3rdHiddenUnit],stddev=0.01))
D_b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit],stddev=0.01))
D_b3 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
D_b4 = tf.Variable(tf.random_normal([n_InputUnit],stddev=0.01))

X = tf.placeholder(tf.float32, [None, n_InputUnit]) # tensorflow의 placeholder라는 변수를 선언
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    # Encoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, E_W1), E_b1))
    layer_2 = tf.nn.sigmoid(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    layer_3 = tf.nn.sigmoid(tf.add( tf.matmul(layer_2, E_W3), E_b3))
    layer_4 = tf.nn.sigmoid(tf.add( tf.matmul(layer_3, E_W4), E_b4))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    return layer_4

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    # Decoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, D_W1), D_b1))
    layer_2 = tf.nn.sigmoid(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    layer_3 = tf.nn.sigmoid(tf.add( tf.matmul(layer_2, D_W3), D_b3))
    layer_4 = tf.nn.sigmoid(tf.add( tf.matmul(layer_3, D_W4), D_b4))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    return layer_4

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.5).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

k=0

In [9]:
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738)) #25개
        
        _, aa = sess.run([optimizer, loss], feed_dict={X: batch})
        
        k = k+1
        if k == 25:
            k=0
            
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, aa))
        if i == num_steps:
            batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738))
            e = sess.run(encoder_op, feed_dict = {X: batch})
            g = sess.run(decoder_op, feed_dict={X: batch})
            print(i)
            print("batch")
            print(batch)
            print("decoded")
            print(g)
            print("encoded")
            print(e)
            print("----------------------------------------------")
    ###################################################
    print("######################################################")
    print("######################################################")
    for i in range(25):
        batch = np.reshape(FTIR_FullSpectrum[1738*(i) : 1738*(i+1)],(1,1738))
        g = sess.run(decoder_op, feed_dict={X: batch})
        print(i)
        print("batch")
        print(batch)
        print("decoded")
        print(g)
        print("encoded")
        print(e)
        print("----------------------------------------------")

Step 1: Minibatch Loss: 0.010242
Step 1000: Minibatch Loss: 0.000092
1000
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.3430936   0.34183404  0.34370121 ...,  0.55140114  0.55147189
   0.55159003]]
encoded
[[ 0.47913438  0.49260843  0.49649453  0.48253918  0.518134    0.48810002
   0.49916479  0.49892157  0.50741982  0.46618238  0.49605268  0.50480431
   0.50273168  0.47583795  0.4881123   0.47940722  0.49917012  0.49688727
   0.49981606  0.49322042  0.50715333  0.45841482  0.47230154  0.5007596
   0.49465823  0.49854079  0.48951733  0.49190727  0.48510543  0.50968063
   0.51178747  0.4609246   0.52143204  0.50288558  0.48999187  0.49001762
   0.49827588  0.48756498  0.49832585  0.52963072  0.54848433  0.49112675
   0.51972729  0.48830009  0.53198725  0.50153458  0.51035666  0.52622324
   0.50092256  0.47106495  0.51204377  0.4994505   0.51293564  0.50935173
   0.4871943   0.51055068  0.50980753  0.50379574  0.49960193  0.51326662
   0.5464043 

In [10]:
OriginData = np.loadtxt("FTIR Top Data.txt")

FTIR_FullSpectrum = OriginData[:,1]

num_steps = 1000
display_step = 1000

FTIR_FullSpectrum

learning_rate = 0.00001

n_InputUnit = 1738
n_1stHiddenUnit = 869
n_2ndHiddenUnit = 434
n_3rdHiddenUnit = 210
n_4thHiddenUnit = 100
n_5thHiddenUnit = 50

E_W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
E_W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
E_W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
E_W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
E_W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_5thHiddenUnit],stddev=0.01))

D_W1 = tf.Variable(tf.random_normal([n_5thHiddenUnit, n_4thHiddenUnit],stddev=0.01))
D_W2 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
D_W3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
D_W4 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_1stHiddenUnit],stddev=0.01))
D_W5 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_InputUnit],stddev=0.01))

########################################################################################

E_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
E_b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit],stddev=0.01))
E_b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit],stddev=0.01))
E_b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit],stddev=0.01))
E_b5 = tf.Variable(tf.random_normal([n_5thHiddenUnit],stddev=0.01))

D_b1 = tf.Variable(tf.random_normal([n_4thHiddenUnit],stddev=0.01))
D_b2 = tf.Variable(tf.random_normal([n_3rdHiddenUnit],stddev=0.01))
D_b3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit],stddev=0.01))
D_b4 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
D_b5 = tf.Variable(tf.random_normal([n_InputUnit],stddev=0.01))

X = tf.placeholder(tf.float32, [None, n_InputUnit]) # tensorflow의 placeholder라는 변수를 선언
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    # Encoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, E_W1), E_b1))
    layer_2 = tf.nn.sigmoid(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    layer_3 = tf.nn.sigmoid(tf.add( tf.matmul(layer_2, E_W3), E_b3))
    layer_4 = tf.nn.sigmoid(tf.add( tf.matmul(layer_3, E_W4), E_b4))
    layer_5 = tf.nn.sigmoid(tf.add( tf.matmul(layer_4, E_W5), E_b5))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    return layer_5

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    # Decoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, D_W1), D_b1))
    layer_2 = tf.nn.sigmoid(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    layer_3 = tf.nn.sigmoid(tf.add( tf.matmul(layer_2, D_W3), D_b3))
    layer_4 = tf.nn.sigmoid(tf.add( tf.matmul(layer_3, D_W4), D_b4))
    layer_5 = tf.nn.sigmoid(tf.add( tf.matmul(layer_4, D_W5), D_b5))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    return layer_5

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.5).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

k=0

In [11]:
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738)) #25개
        
        _, aa = sess.run([optimizer, loss], feed_dict={X: batch})
        
        k = k+1
        if k == 25:
            k=0
            
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, aa))
        if i == num_steps:
            batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738))
            e = sess.run(encoder_op, feed_dict = {X: batch})
            g = sess.run(decoder_op, feed_dict={X: batch})
            print(i)
            print("batch")
            print(batch)
            print("decoded")
            print(g)
            print("encoded")
            print(e)
            print("----------------------------------------------")
    ###################################################
    print("######################################################")
    print("######################################################")
    for i in range(25):
        batch = np.reshape(FTIR_FullSpectrum[1738*(i) : 1738*(i+1)],(1,1738))
        g = sess.run(decoder_op, feed_dict={X: batch})
        print(i)
        print("batch")
        print(batch)
        print("decoded")
        print(g)
        print("encoded")
        print(e)
        print("----------------------------------------------")

Step 1: Minibatch Loss: 0.009954
Step 1000: Minibatch Loss: 0.000092
1000
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.34308785  0.34183234  0.34370029 ...,  0.5514062   0.55147672
   0.55159211]]
encoded
[[ 0.50063205  0.49215969  0.50780791  0.51005071  0.51616222  0.49033946
   0.50278181  0.49676493  0.48435047  0.49853471  0.50256783  0.49271673
   0.49661869  0.50357419  0.52442241  0.48751107  0.49374738  0.51036716
   0.48694685  0.49974328  0.47810262  0.51550215  0.48478448  0.48466763
   0.50249565  0.50198424  0.5083853   0.48565844  0.51093924  0.51050729
   0.52013248  0.52400774  0.50199175  0.50592369  0.49669909  0.5076828
   0.49773967  0.49713686  0.49170619  0.48765013  0.48147142  0.4968299
   0.49680075  0.46485934  0.50818366  0.48314595  0.47602415  0.49849012
   0.50689405  0.50271291]]
----------------------------------------------
######################################################
################################

In [12]:
OriginData = np.loadtxt("FTIR Top Data.txt")

FTIR_FullSpectrum = OriginData[:,1]

num_steps = 1000
display_step = 1000

FTIR_FullSpectrum

learning_rate = 0.00001

n_InputUnit = 1738
n_1stHiddenUnit = 869
n_2ndHiddenUnit = 434
n_3rdHiddenUnit = 210
n_4thHiddenUnit = 100
n_5thHiddenUnit = 50
n_6thHiddenUnit = 25

E_W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
E_W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
E_W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
E_W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
E_W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_5thHiddenUnit],stddev=0.01))
E_W6 = tf.Variable(tf.random_normal([n_5thHiddenUnit, n_6thHiddenUnit],stddev=0.01))

D_W1 = tf.Variable(tf.random_normal([n_6thHiddenUnit, n_5thHiddenUnit],stddev=0.01))
D_W2 = tf.Variable(tf.random_normal([n_5thHiddenUnit, n_4thHiddenUnit],stddev=0.01))
D_W3 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
D_W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
D_W5 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_1stHiddenUnit],stddev=0.01))
D_W6 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_InputUnit],stddev=0.01))

########################################################################################

E_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
E_b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit],stddev=0.01))
E_b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit],stddev=0.01))
E_b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit],stddev=0.01))
E_b5 = tf.Variable(tf.random_normal([n_5thHiddenUnit],stddev=0.01))
E_b6 = tf.Variable(tf.random_normal([n_6thHiddenUnit],stddev=0.01))

D_b1 = tf.Variable(tf.random_normal([n_5thHiddenUnit],stddev=0.01))
D_b2 = tf.Variable(tf.random_normal([n_4thHiddenUnit],stddev=0.01))
D_b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit],stddev=0.01))
D_b4 = tf.Variable(tf.random_normal([n_2ndHiddenUnit],stddev=0.01))
D_b5 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
D_b6 = tf.Variable(tf.random_normal([n_InputUnit],stddev=0.01))

X = tf.placeholder(tf.float32, [None, n_InputUnit]) # tensorflow의 placeholder라는 변수를 선언
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    # Encoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, E_W1), E_b1))
    layer_2 = tf.nn.sigmoid(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    layer_3 = tf.nn.sigmoid(tf.add( tf.matmul(layer_2, E_W3), E_b3))
    layer_4 = tf.nn.sigmoid(tf.add( tf.matmul(layer_3, E_W4), E_b4))
    layer_5 = tf.nn.sigmoid(tf.add( tf.matmul(layer_4, E_W5), E_b5))
    layer_6 = tf.nn.sigmoid(tf.add( tf.matmul(layer_5, E_W6), E_b6))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    return layer_6

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    # Decoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, D_W1), D_b1))
    layer_2 = tf.nn.sigmoid(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    layer_3 = tf.nn.sigmoid(tf.add( tf.matmul(layer_2, D_W3), D_b3))
    layer_4 = tf.nn.sigmoid(tf.add( tf.matmul(layer_3, D_W4), D_b4))
    layer_5 = tf.nn.sigmoid(tf.add( tf.matmul(layer_4, D_W5), D_b5))
    layer_6 = tf.nn.sigmoid(tf.add( tf.matmul(layer_5, D_W6), D_b6))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    return layer_6

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.5).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

k=0

In [13]:
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738)) #25개
        
        _, aa = sess.run([optimizer, loss], feed_dict={X: batch})
        
        k = k+1
        if k == 25:
            k=0
            
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, aa))
        if i == num_steps:
            batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738))
            e = sess.run(encoder_op, feed_dict = {X: batch})
            g = sess.run(decoder_op, feed_dict={X: batch})
            print(i)
            print("batch")
            print(batch)
            print("decoded")
            print(g)
            print("encoded")
            print(e)
            print("----------------------------------------------")
    ###################################################
    print("######################################################")
    print("######################################################")
    for i in range(25):
        batch = np.reshape(FTIR_FullSpectrum[1738*(i) : 1738*(i+1)],(1,1738))
        g = sess.run(decoder_op, feed_dict={X: batch})
        print(i)
        print("batch")
        print(batch)
        print("decoded")
        print(g)
        print("encoded")
        print(e)
        print("----------------------------------------------")

Step 1: Minibatch Loss: 0.010282
Step 1000: Minibatch Loss: 0.000092
1000
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.34308544  0.34182855  0.34369794 ...,  0.55140656  0.55147845
   0.55159515]]
encoded
[[ 0.49831685  0.50307912  0.51399541  0.49815717  0.51515812  0.50139558
   0.49350804  0.51069921  0.50699604  0.50473642  0.51763135  0.49546236
   0.50645494  0.50405586  0.50297773  0.49153379  0.50501847  0.49990848
   0.49966031  0.51936841  0.51346564  0.50201273  0.49696529  0.49137601
   0.50151068]]
----------------------------------------------
######################################################
######################################################
0
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.34308544  0.34182855  0.34369794 ...,  0.55140656  0.55147845
   0.55159515]]
encoded
[[ 0.49831685  0.50307912  0.51399541  0.49815717  0.51515812  0.50139558
   0.49350804  0.51069921  0.5069

In [15]:
OriginData = np.loadtxt("FTIR Top Data.txt")

FTIR_FullSpectrum = OriginData[:,1]

num_steps = 2000
display_step = 1000

FTIR_FullSpectrum

learning_rate = 0.00001

n_InputUnit = 1738
n_1stHiddenUnit = 869

E_W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
D_W1 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_InputUnit],stddev=0.01))

E_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
D_b1 = tf.Variable(tf.random_normal([n_InputUnit],stddev=0.01))

X = tf.placeholder(tf.float32, [None, n_InputUnit]) # tensorflow의 placeholder라는 변수를 선언
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    # Encoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, E_W1), E_b1))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    return layer_1

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    # Decoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, D_W1), D_b1))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    return layer_1

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.5).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

k=0

In [16]:
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738)) #25개
        
        _, aa = sess.run([optimizer, loss], feed_dict={X: batch})
        
        k = k+1
        if k == 25:
            k=0
            
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, aa))
        if i == num_steps:
            batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738))
            e = sess.run(encoder_op, feed_dict = {X: batch})
            g = sess.run(decoder_op, feed_dict={X: batch})
            print(i)
            print("batch")
            print(batch)
            print("decoded")
            print(g)
            print("encoded")
            print(e)
            print("----------------------------------------------")
    ###################################################
    print("######################################################")
    print("######################################################")
    for i in range(25):
        batch = np.reshape(FTIR_FullSpectrum[1738*(i) : 1738*(i+1)],(1,1738))
        g = sess.run(decoder_op, feed_dict={X: batch})
        print(i)
        print("batch")
        print(batch)
        print("decoded")
        print(g)
        print("encoded")
        print(e)
        print("----------------------------------------------")

Step 1: Minibatch Loss: 0.010263
Step 1000: Minibatch Loss: 0.000092
Step 2000: Minibatch Loss: 0.000092
2000
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.34318894  0.3419669   0.34386137 ...,  0.55136764  0.55138284
   0.55153638]]
encoded
[[ 0.51526928  0.52698869  0.53373265  0.56617767  0.56893224  0.47901717
   0.56611753  0.50938809  0.59335828  0.54918581  0.51822799  0.557275
   0.47235769  0.52349144  0.49068192  0.44220036  0.4706451   0.45484471
   0.51190984  0.53606576  0.58349919  0.59059912  0.62127399  0.6950261
   0.41819489  0.54614234  0.53489935  0.46480495  0.48685721  0.5422951
   0.47330448  0.51318371  0.52863342  0.49510711  0.51360607  0.49983937
   0.56122756  0.48710495  0.54427284  0.45170578  0.49023309  0.46649259
   0.5778144   0.39542818  0.50113493  0.54819334  0.38795263  0.56779289
   0.60582048  0.53953058  0.56391436  0.50971395  0.54870057  0.52298063
   0.45941091  0.59476697  0.50573164  0.42872712  0.6

[[ 0.51526928  0.52698869  0.53373265  0.56617767  0.56893224  0.47901717
   0.56611753  0.50938809  0.59335828  0.54918581  0.51822799  0.557275
   0.47235769  0.52349144  0.49068192  0.44220036  0.4706451   0.45484471
   0.51190984  0.53606576  0.58349919  0.59059912  0.62127399  0.6950261
   0.41819489  0.54614234  0.53489935  0.46480495  0.48685721  0.5422951
   0.47330448  0.51318371  0.52863342  0.49510711  0.51360607  0.49983937
   0.56122756  0.48710495  0.54427284  0.45170578  0.49023309  0.46649259
   0.5778144   0.39542818  0.50113493  0.54819334  0.38795263  0.56779289
   0.60582048  0.53953058  0.56391436  0.50971395  0.54870057  0.52298063
   0.45941091  0.59476697  0.50573164  0.42872712  0.64545774  0.53922141
   0.49860355  0.57613766  0.52924103  0.54457581  0.52013576  0.53300643
   0.4306502   0.58665526  0.51672941  0.49832484  0.56259531  0.50062239
   0.52375233  0.61179465  0.52391607  0.52124906  0.49545592  0.51348424
   0.45401943  0.50636005  0.47941175  0.6

[[ 0.51526928  0.52698869  0.53373265  0.56617767  0.56893224  0.47901717
   0.56611753  0.50938809  0.59335828  0.54918581  0.51822799  0.557275
   0.47235769  0.52349144  0.49068192  0.44220036  0.4706451   0.45484471
   0.51190984  0.53606576  0.58349919  0.59059912  0.62127399  0.6950261
   0.41819489  0.54614234  0.53489935  0.46480495  0.48685721  0.5422951
   0.47330448  0.51318371  0.52863342  0.49510711  0.51360607  0.49983937
   0.56122756  0.48710495  0.54427284  0.45170578  0.49023309  0.46649259
   0.5778144   0.39542818  0.50113493  0.54819334  0.38795263  0.56779289
   0.60582048  0.53953058  0.56391436  0.50971395  0.54870057  0.52298063
   0.45941091  0.59476697  0.50573164  0.42872712  0.64545774  0.53922141
   0.49860355  0.57613766  0.52924103  0.54457581  0.52013576  0.53300643
   0.4306502   0.58665526  0.51672941  0.49832484  0.56259531  0.50062239
   0.52375233  0.61179465  0.52391607  0.52124906  0.49545592  0.51348424
   0.45401943  0.50636005  0.47941175  0.6

In [17]:
OriginData = np.loadtxt("FTIR Top Data.txt")

FTIR_FullSpectrum = OriginData[:,1]

num_steps = 20000
display_step = 1000

FTIR_FullSpectrum

learning_rate = 0.00001

n_InputUnit = 1738
n_1stHiddenUnit = 400

E_W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
D_W1 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_InputUnit],stddev=0.01))

E_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
D_b1 = tf.Variable(tf.random_normal([n_InputUnit],stddev=0.01))

X = tf.placeholder(tf.float32, [None, n_InputUnit]) # tensorflow의 placeholder라는 변수를 선언
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    # Encoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, E_W1), E_b1))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    return layer_1

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    # Decoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, D_W1), D_b1))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    return layer_1

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.5).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

k=0

In [18]:
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738)) #25개
        
        _, aa = sess.run([optimizer, loss], feed_dict={X: batch})
        
        k = k+1
        if k == 25:
            k=0
            
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, aa))
        if i == num_steps:
            batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738))
            e = sess.run(encoder_op, feed_dict = {X: batch})
            g = sess.run(decoder_op, feed_dict={X: batch})
            print(i)
            print("batch")
            print(batch)
            print("decoded")
            print(g)
            print("encoded")
            print(e)
            print("----------------------------------------------")
    ###################################################
    print("######################################################")
    print("######################################################")
    for i in range(25):
        batch = np.reshape(FTIR_FullSpectrum[1738*(i) : 1738*(i+1)],(1,1738))
        g = sess.run(decoder_op, feed_dict={X: batch})
        print(i)
        print("batch")
        print(batch)
        print("decoded")
        print(g)
        print("encoded")
        print(e)
        print("----------------------------------------------")

Step 1: Minibatch Loss: 0.009456
Step 1000: Minibatch Loss: 0.000092
Step 2000: Minibatch Loss: 0.000092
Step 3000: Minibatch Loss: 0.000092
Step 4000: Minibatch Loss: 0.000092
Step 5000: Minibatch Loss: 0.000092
Step 6000: Minibatch Loss: 0.000092
Step 7000: Minibatch Loss: 0.000092
Step 8000: Minibatch Loss: 0.000092
Step 9000: Minibatch Loss: 0.000092
Step 10000: Minibatch Loss: 0.000092
Step 11000: Minibatch Loss: 0.000092
Step 12000: Minibatch Loss: 0.000092
Step 13000: Minibatch Loss: 0.000092
Step 14000: Minibatch Loss: 0.000092
Step 15000: Minibatch Loss: 0.000092
Step 16000: Minibatch Loss: 0.000092
Step 17000: Minibatch Loss: 0.000092
Step 18000: Minibatch Loss: 0.000092
Step 19000: Minibatch Loss: 0.000092
Step 20000: Minibatch Loss: 0.000092
20000
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.34298685  0.34176296  0.34359911 ...,  0.5515821   0.55175722
   0.55182385]]
encoded
[[ 0.57169604  0.3552703   0.53056467  0.44500539  0.608

[[ 0.57169604  0.3552703   0.53056467  0.44500539  0.60839325  0.65057564
   0.54551125  0.60144413  0.53781784  0.6116156   0.63141072  0.55051017
   0.46883953  0.60977191  0.54047173  0.57207739  0.59794003  0.54100305
   0.51917154  0.45579737  0.5231995   0.67185169  0.54899329  0.62568402
   0.57991695  0.43233183  0.53974003  0.39695701  0.53597522  0.60190463
   0.55314165  0.61732781  0.5085417   0.62932181  0.61902559  0.53118086
   0.56727004  0.48419032  0.57480299  0.6221683   0.56884444  0.64300805
   0.59908736  0.60841507  0.49754396  0.56904256  0.55784672  0.62959194
   0.60012007  0.64095962  0.68866104  0.52680892  0.60366166  0.49212113
   0.61058825  0.52902651  0.51047903  0.60432619  0.59843087  0.51759583
   0.58088994  0.61766022  0.64990073  0.56616837  0.59500015  0.583588
   0.54948449  0.49015898  0.54945654  0.64349473  0.62381506  0.53488028
   0.58732927  0.58601135  0.62448335  0.49152544  0.56652844  0.48317546
   0.45302111  0.63527203  0.56369913  0

In [19]:
OriginData = np.loadtxt("FTIR Top Data.txt")

FTIR_FullSpectrum = OriginData[:,1]

num_steps = 20000
display_step = 1000

FTIR_FullSpectrum

learning_rate = 0.00001

n_InputUnit = 1738
n_1stHiddenUnit = 200

E_W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
D_W1 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_InputUnit],stddev=0.01))

E_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
D_b1 = tf.Variable(tf.random_normal([n_InputUnit],stddev=0.01))

X = tf.placeholder(tf.float32, [None, n_InputUnit]) # tensorflow의 placeholder라는 변수를 선언
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    # Encoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, E_W1), E_b1))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    return layer_1

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    # Decoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, D_W1), D_b1))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    return layer_1

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.5).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

k=0

In [20]:
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738)) #25개
        
        _, aa = sess.run([optimizer, loss], feed_dict={X: batch})
        
        k = k+1
        if k == 25:
            k=0
            
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, aa))
        if i == num_steps:
            batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738))
            e = sess.run(encoder_op, feed_dict = {X: batch})
            g = sess.run(decoder_op, feed_dict={X: batch})
            print(i)
            print("batch")
            print(batch)
            print("decoded")
            print(g)
            print("encoded")
            print(e)
            print("----------------------------------------------")
    ###################################################
    print("######################################################")
    print("######################################################")
    for i in range(25):
        batch = np.reshape(FTIR_FullSpectrum[1738*(i) : 1738*(i+1)],(1,1738))
        g = sess.run(decoder_op, feed_dict={X: batch})
        print(i)
        print("batch")
        print(batch)
        print("decoded")
        print(g)
        print("encoded")
        print(e)
        print("----------------------------------------------")

Step 1: Minibatch Loss: 0.009120
Step 1000: Minibatch Loss: 0.000094
Step 2000: Minibatch Loss: 0.000093
Step 3000: Minibatch Loss: 0.000093
Step 4000: Minibatch Loss: 0.000093
Step 5000: Minibatch Loss: 0.000093
Step 6000: Minibatch Loss: 0.000093
Step 7000: Minibatch Loss: 0.000093
Step 8000: Minibatch Loss: 0.000093
Step 9000: Minibatch Loss: 0.000093
Step 10000: Minibatch Loss: 0.000093
Step 11000: Minibatch Loss: 0.000093
Step 12000: Minibatch Loss: 0.000093
Step 13000: Minibatch Loss: 0.000093
Step 14000: Minibatch Loss: 0.000093
Step 15000: Minibatch Loss: 0.000093
Step 16000: Minibatch Loss: 0.000093
Step 17000: Minibatch Loss: 0.000093
Step 18000: Minibatch Loss: 0.000093
Step 19000: Minibatch Loss: 0.000093
Step 20000: Minibatch Loss: 0.000093
20000
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.34301081  0.34182152  0.34368649 ...,  0.5516752   0.55175769
   0.55187815]]
encoded
[[ 0.6498661   0.71725988  0.66449094  0.72344869  0.751

[[ 0.6498661   0.71725988  0.66449094  0.72344869  0.75156176  0.60996658
   0.73348624  0.69175631  0.73374099  0.74762076  0.7186622   0.70789206
   0.75876552  0.6715427   0.72763371  0.71992987  0.77890635  0.66555357
   0.73668969  0.68564808  0.72271204  0.68704158  0.70832551  0.73671681
   0.6950888   0.70669764  0.68188137  0.67082083  0.69378358  0.69933909
   0.6944344   0.62071729  0.70961517  0.69470966  0.75622463  0.79299688
   0.72967958  0.7059707   0.67778492  0.54982859  0.67258197  0.69338673
   0.73321563  0.70969343  0.80029398  0.70617473  0.73252982  0.73107803
   0.64528292  0.58822048  0.70020294  0.76958448  0.72073162  0.63428181
   0.73305857  0.6720956   0.65644825  0.77086055  0.66362041  0.73167402
   0.71658576  0.51698357  0.66904896  0.73746568  0.70995945  0.74100512
   0.75430208  0.75437868  0.69340771  0.81124699  0.70504105  0.76943409
   0.7584399   0.69946718  0.70333964  0.70336747  0.74067616  0.61085784
   0.7376374   0.53852111  0.75245166 

In [21]:
OriginData = np.loadtxt("FTIR Top Data.txt")

FTIR_FullSpectrum = OriginData[:,1]

num_steps = 20000
display_step = 1000

FTIR_FullSpectrum

learning_rate = 0.00001

n_InputUnit = 1738
n_1stHiddenUnit = 100

E_W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
D_W1 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_InputUnit],stddev=0.01))

E_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
D_b1 = tf.Variable(tf.random_normal([n_InputUnit],stddev=0.01))

X = tf.placeholder(tf.float32, [None, n_InputUnit]) # tensorflow의 placeholder라는 변수를 선언
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    # Encoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, E_W1), E_b1))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    return layer_1

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    # Decoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, D_W1), D_b1))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    return layer_1

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.5).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

k=0

In [22]:
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738)) #25개
        
        _, aa = sess.run([optimizer, loss], feed_dict={X: batch})
        
        k = k+1
        if k == 25:
            k=0
            
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, aa))
        if i == num_steps:
            batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738))
            e = sess.run(encoder_op, feed_dict = {X: batch})
            g = sess.run(decoder_op, feed_dict={X: batch})
            print(i)
            print("batch")
            print(batch)
            print("decoded")
            print(g)
            print("encoded")
            print(e)
            print("----------------------------------------------")
    ###################################################
    print("######################################################")
    print("######################################################")
    for i in range(25):
        batch = np.reshape(FTIR_FullSpectrum[1738*(i) : 1738*(i+1)],(1,1738))
        g = sess.run(decoder_op, feed_dict={X: batch})
        print(i)
        print("batch")
        print(batch)
        print("decoded")
        print(g)
        print("encoded")
        print(e)
        print("----------------------------------------------")

Step 1: Minibatch Loss: 0.009008
Step 1000: Minibatch Loss: 0.000193
Step 2000: Minibatch Loss: 0.000093
Step 3000: Minibatch Loss: 0.000093
Step 4000: Minibatch Loss: 0.000093
Step 5000: Minibatch Loss: 0.000093
Step 6000: Minibatch Loss: 0.000093
Step 7000: Minibatch Loss: 0.000093
Step 8000: Minibatch Loss: 0.000093
Step 9000: Minibatch Loss: 0.000093
Step 10000: Minibatch Loss: 0.000093
Step 11000: Minibatch Loss: 0.000093
Step 12000: Minibatch Loss: 0.000093
Step 13000: Minibatch Loss: 0.000093
Step 14000: Minibatch Loss: 0.000093
Step 15000: Minibatch Loss: 0.000093
Step 16000: Minibatch Loss: 0.000093
Step 17000: Minibatch Loss: 0.000093
Step 18000: Minibatch Loss: 0.000093
Step 19000: Minibatch Loss: 0.000093
Step 20000: Minibatch Loss: 0.000093
20000
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.34287444  0.34168586  0.3435224  ...,  0.55177915  0.55186611
   0.5519684 ]]
encoded
[[ 0.8678863   0.87788188  0.86744052  0.83249849  0.874

batch
[[ 0.348106  0.342606  0.346808 ...,  0.560818  0.560785  0.560907]]
decoded
[[ 0.34213069  0.34097114  0.34279922 ...,  0.55202508  0.55209553
   0.55222207]]
encoded
[[ 0.8678863   0.87788188  0.86744052  0.83249849  0.87431431  0.84946555
   0.83581775  0.87793523  0.85532039  0.8306101   0.8547231   0.8485626
   0.88167143  0.85727829  0.85654038  0.86747193  0.85997719  0.84491342
   0.83390695  0.84546345  0.8572042   0.80670422  0.75281358  0.8776775
   0.85400349  0.89054328  0.86166936  0.86444926  0.86356258  0.75312144
   0.82319206  0.87018609  0.86507857  0.87725604  0.80547094  0.77873504
   0.78626949  0.81309253  0.88866395  0.85720664  0.86669272  0.84410644
   0.89110863  0.85061663  0.84159827  0.89592522  0.87950176  0.8880105
   0.87071431  0.87966365  0.87850416  0.88525355  0.8634339   0.85327572
   0.88886231  0.88902891  0.87700135  0.87996197  0.88612372  0.8434456
   0.86202866  0.87156016  0.88278061  0.81217486  0.8742891   0.81669432
   0.85807532  0

In [23]:
OriginData = np.loadtxt("FTIR Top Data.txt")

FTIR_FullSpectrum = OriginData[:,1]

num_steps = 20000
display_step = 1000

FTIR_FullSpectrum

learning_rate = 0.00001

n_InputUnit = 1738
n_1stHiddenUnit = 50

E_W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
D_W1 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_InputUnit],stddev=0.01))

E_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
D_b1 = tf.Variable(tf.random_normal([n_InputUnit],stddev=0.01))

X = tf.placeholder(tf.float32, [None, n_InputUnit]) # tensorflow의 placeholder라는 변수를 선언
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    # Encoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, E_W1), E_b1))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    return layer_1

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    # Decoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, D_W1), D_b1))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    return layer_1

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.5).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

k=0

In [24]:
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738)) #25개
        
        _, aa = sess.run([optimizer, loss], feed_dict={X: batch})
        
        k = k+1
        if k == 25:
            k=0
            
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, aa))
        if i == num_steps:
            batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738))
            e = sess.run(encoder_op, feed_dict = {X: batch})
            g = sess.run(decoder_op, feed_dict={X: batch})
            print(i)
            print("batch")
            print(batch)
            print("decoded")
            print(g)
            print("encoded")
            print(e)
            print("----------------------------------------------")
    ###################################################
    print("######################################################")
    print("######################################################")
    for i in range(25):
        batch = np.reshape(FTIR_FullSpectrum[1738*(i) : 1738*(i+1)],(1,1738))
        g = sess.run(decoder_op, feed_dict={X: batch})
        print(i)
        print("batch")
        print(batch)
        print("decoded")
        print(g)
        print("encoded")
        print(e)
        print("----------------------------------------------")

Step 1: Minibatch Loss: 0.008946
Step 1000: Minibatch Loss: 0.001240
Step 2000: Minibatch Loss: 0.000097
Step 3000: Minibatch Loss: 0.000092
Step 4000: Minibatch Loss: 0.000093
Step 5000: Minibatch Loss: 0.000093
Step 6000: Minibatch Loss: 0.000093
Step 7000: Minibatch Loss: 0.000093
Step 8000: Minibatch Loss: 0.000093
Step 9000: Minibatch Loss: 0.000093
Step 10000: Minibatch Loss: 0.000093
Step 11000: Minibatch Loss: 0.000093
Step 12000: Minibatch Loss: 0.000093
Step 13000: Minibatch Loss: 0.000093
Step 14000: Minibatch Loss: 0.000093
Step 15000: Minibatch Loss: 0.000093
Step 16000: Minibatch Loss: 0.000093
Step 17000: Minibatch Loss: 0.000093
Step 18000: Minibatch Loss: 0.000093
Step 19000: Minibatch Loss: 0.000093
Step 20000: Minibatch Loss: 0.000093
20000
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.34255269  0.34139732  0.34321651 ...,  0.5518921   0.55196905
   0.55208731]]
encoded
[[ 0.95984834  0.95206451  0.94731385  0.95769393  0.964

In [25]:
OriginData = np.loadtxt("FTIR Top Data.txt")

FTIR_FullSpectrum = OriginData[:,1]

num_steps = 20000
display_step = 1000

FTIR_FullSpectrum

learning_rate = 0.00001

n_InputUnit = 1738
n_1stHiddenUnit = 25

E_W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
D_W1 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_InputUnit],stddev=0.01))

E_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
D_b1 = tf.Variable(tf.random_normal([n_InputUnit],stddev=0.01))

X = tf.placeholder(tf.float32, [None, n_InputUnit]) # tensorflow의 placeholder라는 변수를 선언
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    # Encoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, E_W1), E_b1))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    return layer_1

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    # Decoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, D_W1), D_b1))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    return layer_1

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.5).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

k=0

In [26]:
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738)) #25개
        
        _, aa = sess.run([optimizer, loss], feed_dict={X: batch})
        
        k = k+1
        if k == 25:
            k=0
            
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, aa))
        if i == num_steps:
            batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738))
            e = sess.run(encoder_op, feed_dict = {X: batch})
            g = sess.run(decoder_op, feed_dict={X: batch})
            print(i)
            print("batch")
            print(batch)
            print("decoded")
            print(g)
            print("encoded")
            print(e)
            print("----------------------------------------------")
    ###################################################
    print("######################################################")
    print("######################################################")
    for i in range(25):
        batch = np.reshape(FTIR_FullSpectrum[1738*(i) : 1738*(i+1)],(1,1738))
        g = sess.run(decoder_op, feed_dict={X: batch})
        print(i)
        print("batch")
        print(batch)
        print("decoded")
        print(g)
        print("encoded")
        print(e)
        print("----------------------------------------------")

Step 1: Minibatch Loss: 0.008984
Step 1000: Minibatch Loss: 0.003469
Step 2000: Minibatch Loss: 0.000625
Step 3000: Minibatch Loss: 0.000125
Step 4000: Minibatch Loss: 0.000092
Step 5000: Minibatch Loss: 0.000092
Step 6000: Minibatch Loss: 0.000092
Step 7000: Minibatch Loss: 0.000092
Step 8000: Minibatch Loss: 0.000092
Step 9000: Minibatch Loss: 0.000092
Step 10000: Minibatch Loss: 0.000092
Step 11000: Minibatch Loss: 0.000092
Step 12000: Minibatch Loss: 0.000092
Step 13000: Minibatch Loss: 0.000092
Step 14000: Minibatch Loss: 0.000092
Step 15000: Minibatch Loss: 0.000092
Step 16000: Minibatch Loss: 0.000092
Step 17000: Minibatch Loss: 0.000092
Step 18000: Minibatch Loss: 0.000092
Step 19000: Minibatch Loss: 0.000092
Step 20000: Minibatch Loss: 0.000092
20000
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.34235862  0.34121087  0.34302697 ...,  0.55197304  0.55205297
   0.55217129]]
encoded
[[ 0.98980486  0.9888255   0.98820168  0.98913926  0.989

In [27]:
OriginData = np.loadtxt("FTIR Top Data.txt")

FTIR_FullSpectrum = OriginData[:,1]

num_steps = 20000
display_step = 1000

FTIR_FullSpectrum

learning_rate = 0.00001

n_InputUnit = 1738
n_1stHiddenUnit = 10

E_W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
D_W1 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_InputUnit],stddev=0.01))

E_b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit],stddev=0.01))
D_b1 = tf.Variable(tf.random_normal([n_InputUnit],stddev=0.01))

X = tf.placeholder(tf.float32, [None, n_InputUnit]) # tensorflow의 placeholder라는 변수를 선언
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    # Encoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, E_W1), E_b1))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, E_W1), E_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, E_W2), E_b2))
    
    return layer_1

def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    # Decoder Hidden layer with sigmoid activation #2
    
    layer_1 = tf.nn.sigmoid(tf.add( tf.matmul(x, D_W1), D_b1))
    
    #layer_1 = tf.nn.relu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.relu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    #layer_1 = tf.nn.elu(tf.add( tf.matmul(x, D_W1), D_b1))
    #layer_2 = tf.nn.elu(tf.add( tf.matmul(layer_1, D_W2), D_b2))
    
    return layer_1

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
#optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.5).minimize(loss)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

k=0

In [32]:
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738)) #25개
        
        _, aa = sess.run([optimizer, loss], feed_dict={X: batch})
        
        k = k+1
        if k == 25:
            k=0
            
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, aa))
        if i == num_steps:
            batch = np.reshape(FTIR_FullSpectrum[1738*(k) : 1738*(k+1)],(1,1738))
            e = sess.run(encoder_op, feed_dict = {X: batch})
            g = sess.run(decoder_op, feed_dict={X: batch})
            print(i)
            print("batch")
            print(batch)
            print("decoded")
            print(g)
            print("encoded")
            print(e)
            print("----------------------------------------------")
    ###################################################
    print("######################################################")
    print("######################################################")
    np.savetxt('AutoEncoder.txt',g)
    
    for i in range(25):
        batch = np.reshape(FTIR_FullSpectrum[1738*(i) : 1738*(i+1)],(1,1738))
        g = sess.run(decoder_op, feed_dict={X: batch})
        print(i)
        print("batch")
        print(batch)
        print("decoded")
        print(g)
        f = open("AutoEncoder.txt",'a')
        f.write(g)
        f.close()
        print("encoded")
        print(e)
        print("----------------------------------------------")
        
    

Step 1: Minibatch Loss: 0.008860
Step 1000: Minibatch Loss: 0.005647
Step 2000: Minibatch Loss: 0.003204
Step 3000: Minibatch Loss: 0.001655
Step 4000: Minibatch Loss: 0.000764
Step 5000: Minibatch Loss: 0.000331
Step 6000: Minibatch Loss: 0.000162
Step 7000: Minibatch Loss: 0.000109
Step 8000: Minibatch Loss: 0.000095
Step 9000: Minibatch Loss: 0.000092
Step 10000: Minibatch Loss: 0.000092
Step 11000: Minibatch Loss: 0.000092
Step 12000: Minibatch Loss: 0.000092
Step 13000: Minibatch Loss: 0.000092
Step 14000: Minibatch Loss: 0.000092
Step 15000: Minibatch Loss: 0.000092
Step 16000: Minibatch Loss: 0.000092
Step 17000: Minibatch Loss: 0.000092
Step 18000: Minibatch Loss: 0.000092
Step 19000: Minibatch Loss: 0.000092
Step 20000: Minibatch Loss: 0.000092
20000
batch
[[ 0.329208  0.329827  0.331068 ...,  0.553091  0.553167  0.553406]]
decoded
[[ 0.34225979  0.34111857  0.34293267 ...,  0.55202597  0.55210257
   0.55222178]]
encoded
[[ 0.99798739  0.99798906  0.99808782  0.99802184  0.998

TypeError: write() argument must be str, not numpy.ndarray